# Sensitivity Analysis: Robustness to Unmeasured Confounding

This notebook covers **sensitivity analysis** methods for assessing how robust causal conclusions are to potential unmeasured confounding.

## Learning Objectives

1. Understand why sensitivity analysis is essential for causal inference
2. Learn the E-value approach for unmeasured confounding
3. Implement Rosenbaum bounds for matched studies
4. Apply sensitivity analysis to biological examples
5. Interpret and report sensitivity analysis results

## Prerequisites

- Treatment effect estimation (`01_treatment_effects.ipynb`)
- Causal graphs and confounding (`02_causal_graphs.ipynb`)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

---

## Part 1: Why Sensitivity Analysis?

### The Fundamental Problem

All causal inference from observational data relies on the **no unmeasured confounding** assumption:

> Given the measured covariates, treatment assignment is independent of potential outcomes.

This assumption is **untestable** from the data alone. We can never prove there isn't an unmeasured confounder.

### What Sensitivity Analysis Does

Instead of assuming no unmeasured confounding, sensitivity analysis asks:

> **How strong would unmeasured confounding need to be to explain away our findings?**

If the answer is "implausibly strong," we gain confidence in our conclusions.

In [ ]:
# Generate example data with known confounding structure
def generate_confounded_data(n=2000, 
                              true_effect=0.5,
                              measured_confounding=0.6,
                              unmeasured_confounding=0.0,
                              seed=42):
    """
    Generate data with both measured and unmeasured confounding.
    
    Parameters
    ----------
    n : int
        Sample size
    true_effect : float
        True causal effect of treatment on outcome
    measured_confounding : float
        Strength of measured confounder (Z)
    unmeasured_confounding : float
        Strength of unmeasured confounder (U)
    
    Returns
    -------
    DataFrame with columns: Z (measured), U (unmeasured), T (treatment), Y (outcome)
    """
    np.random.seed(seed)
    
    # Measured confounder
    Z = np.random.normal(0, 1, n)
    
    # Unmeasured confounder
    U = np.random.normal(0, 1, n)
    
    # Treatment probability depends on both confounders
    logit_T = measured_confounding * Z + unmeasured_confounding * U
    prob_T = 1 / (1 + np.exp(-logit_T))
    T = np.random.binomial(1, prob_T)
    
    # Outcome depends on treatment and both confounders
    Y = (true_effect * T + 
         measured_confounding * Z + 
         unmeasured_confounding * U + 
         np.random.normal(0, 0.5, n))
    
    return pd.DataFrame({'Z': Z, 'U': U, 'T': T, 'Y': Y})

# Generate data with no unmeasured confounding
df_no_unmeasured = generate_confounded_data(unmeasured_confounding=0.0)

# Generate data with unmeasured confounding
df_with_unmeasured = generate_confounded_data(unmeasured_confounding=0.8)

print("Data generated with true effect = 0.5")
print(f"No unmeasured confounding: n={len(df_no_unmeasured)}")
print(f"With unmeasured confounding: n={len(df_with_unmeasured)}")

In [ ]:
# Estimate effects with and without adjusting for unmeasured confounder
def estimate_ate(df, adjust_for):
    """Estimate ATE using linear regression with specified covariates."""
    X = df[['T'] + adjust_for]
    y = df['Y']
    lr = LinearRegression().fit(X, y)
    return lr.coef_[0]  # Coefficient on T

print("Effect Estimates (True effect = 0.5)")
print("="*60)

# No unmeasured confounding case
print("\nCase 1: No unmeasured confounding")
naive_1 = estimate_ate(df_no_unmeasured, [])
adjusted_Z_1 = estimate_ate(df_no_unmeasured, ['Z'])
adjusted_ZU_1 = estimate_ate(df_no_unmeasured, ['Z', 'U'])
print(f"  Naive (no adjustment):     {naive_1:.3f}")
print(f"  Adjusted for Z only:       {adjusted_Z_1:.3f} ✓")
print(f"  Adjusted for Z and U:      {adjusted_ZU_1:.3f} ✓")

# With unmeasured confounding case
print("\nCase 2: With unmeasured confounding (U not measured)")
naive_2 = estimate_ate(df_with_unmeasured, [])
adjusted_Z_2 = estimate_ate(df_with_unmeasured, ['Z'])
adjusted_ZU_2 = estimate_ate(df_with_unmeasured, ['Z', 'U'])
print(f"  Naive (no adjustment):     {naive_2:.3f}")
print(f"  Adjusted for Z only:       {adjusted_Z_2:.3f} ← BIASED (U not measured)")
print(f"  Adjusted for Z and U:      {adjusted_ZU_2:.3f} ✓ (oracle)")

print("\n⚠️  When U is unmeasured, adjusting for Z alone leaves residual bias!")

---

## Part 2: The E-value

The **E-value** (VanderWeele & Ding, 2017) quantifies the minimum strength of association that an unmeasured confounder would need to have with both treatment and outcome to fully explain away an observed effect.

### Definition

For a risk ratio RR, the E-value is:

$$E = RR + \sqrt{RR \times (RR - 1)}$$

### Interpretation

The E-value is the minimum risk ratio that an unmeasured confounder would need to have with BOTH:
1. The treatment (given measured covariates)
2. The outcome (given treatment and measured covariates)

to explain away the observed association.

In [ ]:
def compute_e_value(rr):
    """
    Compute the E-value for a given risk ratio.
    
    Parameters
    ----------
    rr : float
        Risk ratio (must be >= 1; if < 1, use 1/rr)
    
    Returns
    -------
    float : E-value
    """
    if rr < 1:
        rr = 1 / rr
    return rr + np.sqrt(rr * (rr - 1))

def compute_e_value_for_ci(rr, rr_lower):
    """
    Compute E-value for point estimate and confidence interval bound.
    
    Parameters
    ----------
    rr : float
        Point estimate of risk ratio
    rr_lower : float
        Lower bound of confidence interval (closer to null)
    
    Returns
    -------
    tuple : (E-value for point estimate, E-value for CI bound)
    """
    e_point = compute_e_value(rr)
    
    # For CI bound, if it crosses 1, E-value is 1
    if rr_lower <= 1:
        e_ci = 1.0
    else:
        e_ci = compute_e_value(rr_lower)
    
    return e_point, e_ci

# Example E-values for different effect sizes
print("E-values for Different Risk Ratios")
print("="*50)
print(f"{'Risk Ratio':<15} {'E-value':<15} {'Interpretation'}")
print("-"*50)

for rr in [1.5, 2.0, 3.0, 5.0, 10.0]:
    e = compute_e_value(rr)
    print(f"{rr:<15.1f} {e:<15.2f} Confounder needs RR≥{e:.1f} with both T and Y")

print("\nInterpretation: Higher E-values = more robust to unmeasured confounding")

In [ ]:
# Visualize E-value as a function of risk ratio
rr_range = np.linspace(1.01, 10, 100)
e_values = [compute_e_value(rr) for rr in rr_range]

plt.figure(figsize=(10, 6))
plt.plot(rr_range, e_values, 'b-', linewidth=2)
plt.xlabel('Observed Risk Ratio', fontsize=12)
plt.ylabel('E-value', fontsize=12)
plt.title('E-value as a Function of Observed Risk Ratio', fontsize=14)

# Add reference lines
for rr, label in [(2, 'RR=2'), (3, 'RR=3'), (5, 'RR=5')]:
    e = compute_e_value(rr)
    plt.axvline(rr, color='gray', linestyle='--', alpha=0.5)
    plt.axhline(e, color='gray', linestyle='--', alpha=0.5)
    plt.plot(rr, e, 'ro', markersize=8)
    plt.annotate(f'{label}\nE={e:.1f}', (rr+0.1, e+0.3), fontsize=10)

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# E-value contour plot: What combinations of confounder-treatment and confounder-outcome
# associations could explain away an observed effect?

def bias_factor(rr_ut, rr_uy):
    """
    Compute the maximum bias factor from an unmeasured confounder.
    
    Parameters
    ----------
    rr_ut : float
        Risk ratio of U-T association
    rr_uy : float
        Risk ratio of U-Y association
    
    Returns
    -------
    float : Maximum bias factor
    """
    return (rr_ut * rr_uy) / (rr_ut + rr_uy - 1)

# Create contour plot
rr_ut_range = np.linspace(1.1, 5, 50)
rr_uy_range = np.linspace(1.1, 5, 50)
RR_UT, RR_UY = np.meshgrid(rr_ut_range, rr_uy_range)
BIAS = bias_factor(RR_UT, RR_UY)

plt.figure(figsize=(10, 8))
contour = plt.contourf(RR_UT, RR_UY, BIAS, levels=20, cmap='RdYlBu_r')
plt.colorbar(contour, label='Bias Factor')

# Add contour lines for specific bias factors
for bias_level in [1.5, 2.0, 3.0]:
    cs = plt.contour(RR_UT, RR_UY, BIAS, levels=[bias_level], colors='black', linewidths=2)
    plt.clabel(cs, inline=True, fontsize=10, fmt=f'Bias={bias_level}')

plt.xlabel('RR(U→T): Confounder-Treatment Association', fontsize=12)
plt.ylabel('RR(U→Y): Confounder-Outcome Association', fontsize=12)
plt.title('Bias Factor from Unmeasured Confounding\n(Combinations that could explain away observed effects)', fontsize=14)

plt.tight_layout()
plt.show()

print("Interpretation:")
print("- If observed RR=2, any combination in the 'Bias=2' contour could explain it away")
print("- Larger observed effects require stronger confounding to explain away")

---

## Part 3: Rosenbaum Bounds

**Rosenbaum bounds** (Rosenbaum, 2002) provide a different approach to sensitivity analysis, particularly useful for matched studies.

### The Idea

In a matched study, we assume that matched pairs have the same probability of treatment. Rosenbaum bounds ask:

> How much could treatment probabilities differ within matched pairs (due to unmeasured confounding) before our conclusions change?

### The Γ Parameter

Γ (gamma) represents the maximum odds ratio of treatment assignment between two units with the same observed covariates:

$$\frac{1}{\Gamma} \leq \frac{P(T=1|X,U)/(1-P(T=1|X,U))}{P(T=1|X,U')/(1-P(T=1|X,U'))} \leq \Gamma$$

- Γ = 1: No unmeasured confounding (perfect randomization within pairs)
- Γ > 1: Some unmeasured confounding allowed

In [ ]:
def rosenbaum_bounds(treated_outcomes, control_outcomes, gamma_values):
    """
    Compute Rosenbaum bounds for a matched pair study.
    
    Uses the Wilcoxon signed-rank test framework.
    
    Parameters
    ----------
    treated_outcomes : array
        Outcomes for treated units in matched pairs
    control_outcomes : array
        Outcomes for control units in matched pairs
    gamma_values : array
        Values of Γ to evaluate
    
    Returns
    -------
    DataFrame with p-value bounds for each Γ
    """
    # Compute pair differences
    differences = treated_outcomes - control_outcomes
    n_pairs = len(differences)
    
    # Rank absolute differences
    abs_diff = np.abs(differences)
    ranks = stats.rankdata(abs_diff)
    
    # Signs of differences
    signs = np.sign(differences)
    
    # Observed test statistic (sum of positive ranks)
    T_obs = np.sum(ranks[signs > 0])
    
    results = []
    
    for gamma in gamma_values:
        # Under Γ, the probability of a positive difference ranges from
        # 1/(1+Γ) to Γ/(1+Γ)
        p_plus_lower = 1 / (1 + gamma)
        p_plus_upper = gamma / (1 + gamma)
        
        # Expected value and variance of T under null with bias
        # (simplified approximation)
        E_T_lower = np.sum(ranks) * p_plus_lower
        E_T_upper = np.sum(ranks) * p_plus_upper
        
        Var_T = np.sum(ranks**2) * p_plus_upper * (1 - p_plus_upper)
        
        # Z-scores for bounds
        z_lower = (T_obs - E_T_upper) / np.sqrt(Var_T)
        z_upper = (T_obs - E_T_lower) / np.sqrt(Var_T)
        
        # P-values (one-sided, upper tail)
        p_lower = 1 - stats.norm.cdf(z_lower)
        p_upper = 1 - stats.norm.cdf(z_upper)
        
        results.append({
            'gamma': gamma,
            'p_lower': max(0, min(1, p_lower)),
            'p_upper': max(0, min(1, p_upper))
        })
    
    return pd.DataFrame(results)

# Generate matched pair data
np.random.seed(42)
n_pairs = 100

# True treatment effect
true_effect = 0.5

# Matched pair outcomes
baseline = np.random.normal(0, 1, n_pairs)
treated = baseline + true_effect + np.random.normal(0, 0.5, n_pairs)
control = baseline + np.random.normal(0, 0.5, n_pairs)

# Compute bounds
gamma_values = np.arange(1.0, 3.1, 0.1)
bounds = rosenbaum_bounds(treated, control, gamma_values)

print("Rosenbaum Bounds for Matched Pair Study")
print("="*50)
print(bounds[bounds['gamma'].isin([1.0, 1.5, 2.0, 2.5, 3.0])].to_string(index=False))

In [ ]:
# Visualize Rosenbaum bounds
plt.figure(figsize=(10, 6))

plt.fill_between(bounds['gamma'], bounds['p_lower'], bounds['p_upper'], 
                 alpha=0.3, color='blue', label='P-value range')
plt.plot(bounds['gamma'], bounds['p_lower'], 'b-', linewidth=2, label='Lower bound')
plt.plot(bounds['gamma'], bounds['p_upper'], 'b--', linewidth=2, label='Upper bound')

# Reference line at p=0.05
plt.axhline(0.05, color='red', linestyle=':', linewidth=2, label='α = 0.05')

# Find critical Γ where lower bound crosses 0.05
critical_idx = np.where(bounds['p_upper'] > 0.05)[0]
if len(critical_idx) > 0:
    critical_gamma = bounds['gamma'].iloc[critical_idx[0]]
    plt.axvline(critical_gamma, color='green', linestyle='--', alpha=0.7)
    plt.annotate(f'Critical Γ ≈ {critical_gamma:.1f}', 
                 (critical_gamma + 0.1, 0.3), fontsize=12, color='green')

plt.xlabel('Γ (Sensitivity Parameter)', fontsize=12)
plt.ylabel('P-value', fontsize=12)
plt.title('Rosenbaum Sensitivity Analysis\nHow much unmeasured confounding to change conclusions?', fontsize=14)
plt.legend(loc='upper left')
plt.ylim(0, 1)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nInterpretation:")
print(f"- At Γ=1 (no unmeasured confounding), p-value is very small")
print(f"- Results remain significant until Γ ≈ {critical_gamma:.1f}")
print(f"- An unmeasured confounder would need to change treatment odds by {critical_gamma:.1f}x")
print(f"  within matched pairs to explain away the effect")

---

## Part 4: Sensitivity Analysis for Linear Models

For continuous outcomes with linear models, we can derive explicit formulas for how unmeasured confounding affects estimates.

In [ ]:
def linear_sensitivity_analysis(df, treatment_col, outcome_col, covariate_cols,
                                 r2_tu_values, r2_uy_values):
    """
    Sensitivity analysis for linear regression.
    
    Computes adjusted estimates for different strengths of unmeasured confounding,
    parameterized by partial R² values.
    
    Parameters
    ----------
    df : DataFrame
        Data
    treatment_col : str
        Name of treatment column
    outcome_col : str
        Name of outcome column
    covariate_cols : list
        Names of measured covariate columns
    r2_tu_values : array
        Partial R² of U on T (given covariates)
    r2_uy_values : array
        Partial R² of U on Y (given T and covariates)
    
    Returns
    -------
    DataFrame with adjusted estimates for each (r2_tu, r2_uy) combination
    """
    # Fit observed model
    X = df[[treatment_col] + covariate_cols]
    y = df[outcome_col]
    lr = LinearRegression().fit(X, y)
    
    # Observed estimate
    beta_obs = lr.coef_[0]
    
    # Residual variance
    y_pred = lr.predict(X)
    sigma2_y = np.var(y - y_pred)
    
    # Residual variance of T given covariates
    if covariate_cols:
        lr_t = LinearRegression().fit(df[covariate_cols], df[treatment_col])
        t_resid = df[treatment_col] - lr_t.predict(df[covariate_cols])
    else:
        t_resid = df[treatment_col] - df[treatment_col].mean()
    sigma2_t = np.var(t_resid)
    
    results = []
    
    for r2_tu in r2_tu_values:
        for r2_uy in r2_uy_values:
            # Bias formula (Cinelli & Hazlett, 2020)
            # bias = sqrt(r2_tu * r2_uy) * sigma_y / sigma_t * sign
            # (simplified version assuming positive confounding)
            
            bias = np.sqrt(r2_tu * r2_uy * sigma2_y / sigma2_t)
            
            # Adjusted estimate (assuming positive confounding)
            beta_adj_lower = beta_obs - bias
            beta_adj_upper = beta_obs + bias
            
            results.append({
                'r2_tu': r2_tu,
                'r2_uy': r2_uy,
                'beta_obs': beta_obs,
                'bias': bias,
                'beta_adj_lower': beta_adj_lower,
                'beta_adj_upper': beta_adj_upper
            })
    
    return pd.DataFrame(results)

# Apply to our data
r2_values = np.array([0.01, 0.05, 0.10, 0.15, 0.20])

sensitivity_results = linear_sensitivity_analysis(
    df_with_unmeasured,
    treatment_col='T',
    outcome_col='Y',
    covariate_cols=['Z'],
    r2_tu_values=r2_values,
    r2_uy_values=r2_values
)

print("Sensitivity Analysis Results")
print("="*70)
print(f"Observed estimate (adjusting for Z only): {sensitivity_results['beta_obs'].iloc[0]:.3f}")
print(f"True effect: 0.5")
print("\nAdjusted estimates for different unmeasured confounding strengths:")
print(sensitivity_results[['r2_tu', 'r2_uy', 'bias', 'beta_adj_lower']].head(10).to_string(index=False))

In [ ]:
# Contour plot of adjusted estimates
r2_fine = np.linspace(0.01, 0.25, 50)
R2_TU, R2_UY = np.meshgrid(r2_fine, r2_fine)

# Compute bias for each combination
beta_obs = sensitivity_results['beta_obs'].iloc[0]
sigma2_y = 0.5**2  # Approximate
sigma2_t = 0.25  # Approximate

BIAS = np.sqrt(R2_TU * R2_UY * sigma2_y / sigma2_t)
BETA_ADJ = beta_obs - BIAS

plt.figure(figsize=(10, 8))
contour = plt.contourf(R2_TU, R2_UY, BETA_ADJ, levels=20, cmap='RdYlGn')
plt.colorbar(contour, label='Adjusted Effect Estimate')

# Add contour lines
cs = plt.contour(R2_TU, R2_UY, BETA_ADJ, levels=[0, 0.5], colors=['red', 'blue'], linewidths=2)
plt.clabel(cs, inline=True, fontsize=10, fmt='%.1f')

plt.xlabel('Partial R² of U on T (given Z)', fontsize=12)
plt.ylabel('Partial R² of U on Y (given T, Z)', fontsize=12)
plt.title('Sensitivity Analysis: Adjusted Effect Estimates\n(Red line = effect becomes 0)', fontsize=14)

plt.tight_layout()
plt.show()

print("Interpretation:")
print("- Green region: Effect remains positive after adjustment")
print("- Red region: Effect becomes negative (sign reversal)")
print("- Red contour line: Combinations that would explain away the effect entirely")

---

## Part 5: Biological Application - Drug Response Study

Let's apply sensitivity analysis to a realistic biological scenario.

In [ ]:
# Simulate a drug response study with potential unmeasured confounding
def simulate_drug_study(n=1000, seed=42):
    """
    Simulate a drug response study.
    
    Measured covariates: age, baseline_expression
    Unmeasured confounder: genetic_variant (affects both drug metabolism and response)
    Treatment: drug_treatment
    Outcome: tumor_response
    """
    np.random.seed(seed)
    
    # Measured covariates
    age = np.random.normal(55, 10, n)
    baseline_expression = np.random.normal(0, 1, n)
    
    # Unmeasured confounder (genetic variant affecting drug metabolism)
    genetic_variant = np.random.binomial(1, 0.3, n)  # 30% have variant
    
    # Treatment assignment (not randomized - depends on covariates)
    # Doctors more likely to prescribe to younger patients with higher expression
    # Genetic variant also affects prescribing (pharmacogenomics)
    logit_treat = (-0.03 * (age - 55) + 
                   0.5 * baseline_expression + 
                   0.8 * genetic_variant)  # Unmeasured confounding
    prob_treat = 1 / (1 + np.exp(-logit_treat))
    drug_treatment = np.random.binomial(1, prob_treat)
    
    # Outcome (tumor response)
    # True drug effect = 2.0
    # Genetic variant also affects response (independent of drug)
    tumor_response = (2.0 * drug_treatment +  # True causal effect
                      -0.02 * (age - 55) +
                      0.3 * baseline_expression +
                      1.5 * genetic_variant +  # Unmeasured confounding
                      np.random.normal(0, 1, n))
    
    return pd.DataFrame({
        'age': age,
        'baseline_expression': baseline_expression,
        'genetic_variant': genetic_variant,  # Unmeasured in practice
        'drug_treatment': drug_treatment,
        'tumor_response': tumor_response
    })

df_drug = simulate_drug_study()

print("Drug Response Study")
print("="*50)
print(f"Sample size: {len(df_drug)}")
print(f"Treated: {df_drug['drug_treatment'].sum()}")
print(f"Control: {(1 - df_drug['drug_treatment']).sum()}")
print(f"\nTrue causal effect of drug: 2.0")

In [ ]:
# Estimate effects with different adjustment sets
print("Effect Estimates")
print("="*60)

# Naive
lr_naive = LinearRegression().fit(
    df_drug[['drug_treatment']], 
    df_drug['tumor_response']
)
print(f"Naive (no adjustment): {lr_naive.coef_[0]:.3f}")

# Adjusted for measured covariates
lr_measured = LinearRegression().fit(
    df_drug[['drug_treatment', 'age', 'baseline_expression']], 
    df_drug['tumor_response']
)
print(f"Adjusted for age, expression: {lr_measured.coef_[0]:.3f}")

# Oracle (adjusted for unmeasured too)
lr_oracle = LinearRegression().fit(
    df_drug[['drug_treatment', 'age', 'baseline_expression', 'genetic_variant']], 
    df_drug['tumor_response']
)
print(f"Oracle (+ genetic variant): {lr_oracle.coef_[0]:.3f}")

print(f"\nTrue effect: 2.0")
print(f"\nBias from unmeasured confounding: {lr_measured.coef_[0] - 2.0:.3f}")

In [ ]:
# Compute E-value for the drug study
# First, convert to risk ratio scale (approximate)

# For continuous outcomes, we can use the "approximate RR" approach
# RR ≈ exp(β / SD(Y))

beta_obs = lr_measured.coef_[0]
sd_y = df_drug['tumor_response'].std()

# Approximate RR
rr_approx = np.exp(beta_obs / sd_y)

# E-value
e_value = compute_e_value(rr_approx)

print("E-value Analysis for Drug Study")
print("="*50)
print(f"Observed effect (β): {beta_obs:.3f}")
print(f"SD of outcome: {sd_y:.3f}")
print(f"Approximate RR: {rr_approx:.2f}")
print(f"E-value: {e_value:.2f}")
print(f"\nInterpretation:")
print(f"An unmeasured confounder would need RR ≥ {e_value:.1f} with both")
print(f"treatment and outcome to explain away this effect.")

In [ ]:
# Compare to known confounders as benchmarks
print("Benchmarking Against Known Confounders")
print("="*60)

# Effect of age on treatment
lr_age_t = LogisticRegression().fit(
    df_drug[['age']], 
    df_drug['drug_treatment']
)
or_age_t = np.exp(lr_age_t.coef_[0][0] * 10)  # Per 10 years

# Effect of age on outcome
lr_age_y = LinearRegression().fit(
    df_drug[['age', 'drug_treatment']], 
    df_drug['tumor_response']
)
beta_age_y = lr_age_y.coef_[0] * 10  # Per 10 years

print(f"Age (per 10 years):")
print(f"  OR for treatment: {or_age_t:.2f}")
print(f"  Effect on outcome: {beta_age_y:.3f}")

# Effect of baseline expression on treatment
lr_expr_t = LogisticRegression().fit(
    df_drug[['baseline_expression']], 
    df_drug['drug_treatment']
)
or_expr_t = np.exp(lr_expr_t.coef_[0][0])

# Effect of baseline expression on outcome
lr_expr_y = LinearRegression().fit(
    df_drug[['baseline_expression', 'drug_treatment']], 
    df_drug['tumor_response']
)
beta_expr_y = lr_expr_y.coef_[0]

print(f"\nBaseline expression (per SD):")
print(f"  OR for treatment: {or_expr_t:.2f}")
print(f"  Effect on outcome: {beta_expr_y:.3f}")

print(f"\nE-value required: {e_value:.2f}")
print(f"\nConclusion: An unmeasured confounder would need to be MUCH stronger")
print(f"than age or baseline expression to explain away the drug effect.")

---

## Part 6: Reporting Sensitivity Analysis

### Best Practices

1. **Always report sensitivity analysis** for observational causal claims
2. **Use multiple methods** (E-value, Rosenbaum bounds, etc.)
3. **Benchmark against measured confounders** - is the required confounding plausible?
4. **Be honest about limitations** - sensitivity analysis doesn't prove causation

In [ ]:
def sensitivity_analysis_report(effect_estimate, se, outcome_sd, 
                                 measured_confounders=None,
                                 alpha=0.05):
    """
    Generate a comprehensive sensitivity analysis report.
    
    Parameters
    ----------
    effect_estimate : float
        Point estimate of causal effect
    se : float
        Standard error of estimate
    outcome_sd : float
        Standard deviation of outcome
    measured_confounders : list of dicts, optional
        Each dict has 'name', 'or_treatment', 'effect_outcome'
    alpha : float
        Significance level
    """
    # Confidence interval
    z = stats.norm.ppf(1 - alpha/2)
    ci_lower = effect_estimate - z * se
    ci_upper = effect_estimate + z * se
    
    # Approximate RR
    rr_point = np.exp(effect_estimate / outcome_sd)
    rr_lower = np.exp(ci_lower / outcome_sd)
    
    # E-values
    e_point, e_ci = compute_e_value_for_ci(rr_point, rr_lower)
    
    print("="*70)
    print("SENSITIVITY ANALYSIS REPORT")
    print("="*70)
    
    print("\n1. EFFECT ESTIMATE")
    print("-"*40)
    print(f"   Point estimate: {effect_estimate:.3f}")
    print(f"   Standard error: {se:.3f}")
    print(f"   95% CI: [{ci_lower:.3f}, {ci_upper:.3f}]")
    
    print("\n2. E-VALUE ANALYSIS")
    print("-"*40)
    print(f"   Approximate RR: {rr_point:.2f}")
    print(f"   E-value (point estimate): {e_point:.2f}")
    print(f"   E-value (CI bound): {e_ci:.2f}")
    print(f"\n   Interpretation:")
    print(f"   To explain away the point estimate, an unmeasured confounder")
    print(f"   would need RR ≥ {e_point:.1f} with both treatment and outcome.")
    if e_ci > 1:
        print(f"   To move the CI to include the null, RR ≥ {e_ci:.1f} is needed.")
    else:
        print(f"   The CI already includes the null.")
    
    if measured_confounders:
        print("\n3. BENCHMARK COMPARISONS")
        print("-"*40)
        print(f"   {'Confounder':<20} {'OR(T)':<10} {'Effect(Y)':<10}")
        for conf in measured_confounders:
            print(f"   {conf['name']:<20} {conf['or_treatment']:<10.2f} {conf['effect_outcome']:<10.3f}")
        print(f"\n   Required E-value: {e_point:.2f}")
    
    print("\n4. CONCLUSION")
    print("-"*40)
    if e_point > 3:
        print("   The effect appears ROBUST to unmeasured confounding.")
        print("   A very strong confounder would be needed to explain it away.")
    elif e_point > 2:
        print("   The effect has MODERATE robustness to unmeasured confounding.")
        print("   A moderately strong confounder could potentially explain it.")
    else:
        print("   The effect has LIMITED robustness to unmeasured confounding.")
        print("   Even weak confounders could potentially explain it away.")
    
    print("\n" + "="*70)

# Generate report for drug study
sensitivity_analysis_report(
    effect_estimate=lr_measured.coef_[0],
    se=0.15,  # Approximate SE
    outcome_sd=df_drug['tumor_response'].std(),
    measured_confounders=[
        {'name': 'Age (per 10 yrs)', 'or_treatment': or_age_t, 'effect_outcome': beta_age_y},
        {'name': 'Expression (per SD)', 'or_treatment': or_expr_t, 'effect_outcome': beta_expr_y}
    ]
)

---

## Summary

### Key Concepts

1. **Sensitivity analysis** quantifies robustness to unmeasured confounding
2. **E-value** gives minimum confounder strength to explain away an effect
3. **Rosenbaum bounds** show how much hidden bias is tolerable in matched studies
4. **Benchmarking** against measured confounders helps assess plausibility

### Practical Guidelines

- Always perform sensitivity analysis for observational causal claims
- Report E-values for both point estimates and confidence intervals
- Compare required confounding to measured confounders
- Be honest: sensitivity analysis doesn't prove causation

### Limitations

- Sensitivity analysis assumes a specific form of confounding
- Multiple unmeasured confounders may have complex interactions
- "Robust" doesn't mean "causal" - it means "hard to explain away"

### References

- VanderWeele & Ding (2017). "Sensitivity Analysis in Observational Research"
- Rosenbaum (2002). "Observational Studies"
- Cinelli & Hazlett (2020). "Making Sense of Sensitivity"

---

## Exercises

1. **E-value interpretation**: A study finds RR=1.8 for a drug effect. Compute the E-value and interpret it.

2. **Benchmarking**: In your own research, identify 2-3 measured confounders. How strong are their associations with treatment and outcome? Would an unmeasured confounder need to be stronger?

3. **Simulation study**: Generate data with known unmeasured confounding. Apply sensitivity analysis and verify it correctly identifies the bias.

4. **Reporting**: Write a sensitivity analysis paragraph for a hypothetical observational study in your field.